In [37]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import re
import numpy as np



In [38]:
url_1 = 'https://startuptalky.com/indian-startups-funding-investor-data-2024/'
url_2 = 'https://startuptalky.com/indian-startups-funding-investors-data-2025/'

In [39]:
def scrap_data(url,year):

    response = requests.get(url)
    soup = BeautifulSoup(response.content, 'html.parser')
    all_rows = []
    headers = None

    for heading in soup.find_all('h2'):
      table = heading.find_next('table')
      if not table:
          continue

      # Extract month from heading text
      heading_text = heading.get_text(strip=True)

      # Example: "Indian Startup Funding - November 2024 [18 - 23 Nov]"
      # Extract just "November 2024"
      match = re.search(r'([A-Za-z]+) (\d{4})', heading_text)

      if match :
          month = match.group(1)  # First capture group
          year = match.group(2)   # Second capture group

      else :
          month = "Unknown"
          year = "Unknown"


      # Extract headers if not already done
      if not headers:
          headers = [th.text.strip() for th in table.find_all('th')]
          headers.append("Month")  # Add month
          headers.append("Year")   # Add year

      # Extract table rows
      for tr in table.find_all('tr')[1:]:
          cells = [
          td.get_text(strip=True)
          for td in tr.find_all(['td', 'th'])
          if td.get_text(strip=True)  # Only include if not empt
          ]
          if cells:
              cells.append(month)
              cells.append(year)
              if(len(cells)==8):
                all_rows.append(cells)

    df = pd.DataFrame(all_rows, columns=headers if headers else None)

    df.to_csv(f"Startup_Funding_{year}.csv", index=False)

In [40]:
scrap_data(url_1,2024)
scrap_data(url_2,2025)

In [41]:
df_2024 = pd.read_csv('Startup_Funding_2024.csv')
df_2025 = pd.read_csv('Startup_Funding_2025.csv')

df_combined = pd.concat([df_2024, df_2025], ignore_index=True)

In [50]:
df_2025.shape

(471, 8)

In [42]:
df_combined.shape

(876, 8)

In [43]:
# def convert_complex_amount_to_inr(amount_str):
#     if pd.isna(amount_str) or not isinstance(amount_str, str):
#         return np.nan

#     amount_str = amount_str.lower().strip()
#     amount_str = amount_str.replace(',', '')

#     # Define multipliers
#     multipliers = {
#         'k': 1e3,
#         'thousand': 1e3,
#         'lakh': 1e5,
#         'lac': 1e5,
#         'm': 1e6,
#         'mn': 1e6,
#         'million': 1e6,
#         'b': 1e9,
#         'billion': 1e9,
#         'cr': 1e7,
#         'crore': 1e7
#     }

#     usd_to_inr = 83  # conversion rate

#     def extract_amount(text, is_usd=False):
#         matches = re.findall(r'([\d\.]+)\s*([a-zA-Z]+)', text)
#         for num, suffix in matches:
#             try:
#                 multiplier = multipliers.get(suffix, 1)
#                 amount = float(num) * multiplier
#                 if is_usd:
#                     amount *= usd_to_inr
#                 return amount
#             except:
#                 continue
#         return None

#     # Try INR first
#     if 'rs' in amount_str or '₹' in amount_str or 'inr' in amount_str or 'Rs' in amount_str:
#         inr_amount = extract_amount(amount_str, is_usd=False)
#         if inr_amount:
#             return round(inr_amount, 2)

#     # Try USD fallback
#     if '$' in amount_str or 'usd' in amount_str:
#         usd_amount = extract_amount(amount_str, is_usd=True)
#         if usd_amount:
#             return round(usd_amount, 2)

#     return np.nan

In [44]:
def convert_complex_amount_to_inr(amount_str):
    if pd.isna(amount_str) or not isinstance(amount_str, str):
        return np.nan

    amount_str = amount_str.lower().strip().replace(',', '')

    # Multipliers
    multipliers = {
        'k': 1e3, 'thousand': 1e3,
        'lakh': 1e5, 'lac': 1e5,
        'm': 1e6, 'mn': 1e6, 'million': 1e6,
        'b': 1e9, 'billion': 1e9,
        'cr': 1e7, 'crore': 1e7
    }

    usd_to_inr = 83

    # Extract all number + suffix matches
    matches = re.findall(r'([\$₹]?\s*\d+(?:\.\d+)?)\s*([a-zA-Z]+)?', amount_str)

    inr_amounts = []
    usd_amounts = []

    for full, suffix in matches:
        # Clean number and currency
        currency = ''
        num_str = full.strip()

        if num_str.startswith('$'):
            currency = 'usd'
            num_str = num_str[1:]
        elif 'usd' in amount_str:
            currency = 'usd'
        elif any(x in amount_str for x in ['rs', 'inr', '₹']):
            currency = 'inr'

        try:
            num = float(num_str)
            multiplier = multipliers.get(suffix or '', 1)
            amount = num * multiplier

            if currency == 'usd':
                usd_amounts.append(amount * usd_to_inr)
            else:
                inr_amounts.append(amount)
        except:
            continue

    # Prefer INR values if both present, else fallback
    if inr_amounts:
        return round(max(inr_amounts), 2)
    elif usd_amounts:
        return round(max(usd_amounts), 2)
    else:
        return np.nan

In [45]:
df_combined['Amount_INR'] = df_combined['Amount'].apply(convert_complex_amount_to_inr)

In [46]:
df_combined


,Company,Sector,Headquarters,Amount,Funding Round Type,Lead Investors,Month,Year,Amount_INR
0,Zaggle Prepaid Ocean Services,Enterprise Fintech,Hyderabad,Rs 595 crore,Funding Round,Qualified Institutional Placement (Bank of Ind...,December,2024,5.950000e+09
1,EPACK Prefab,Pre-engineered Buildings,Greater Noida,$20 million,First Funding Round,GEF Capital Partners,December,2024,1.660000e+09
2,Captain Fresh,Seafood B2B,Bengaluru,Rs 100 crore (~$12 million),Funding Round,Motilal Oswal Wealth Limited,December,2024,1.000000e+09
3,Univest,Retail Advisory Platform,Gurugram,$10 million,Series A,Bertelsmann India Investments (BII),December,2024,8.300000e+08
4,Mindgrove Technologies,Semiconductor Design,Chennai,$8 million,Series A,"Rocketship.vc, Speciale Invest, Mela Ventures,...",December,2024,6.640000e+08
...,...,...,...,...,...,...,...,...,...
871,Zoplar,Medical Equipment Procurement,Gurugram,$3.4 million,Series A,"Blume Ventures, Beenext, Saison Capital, Atriu...",January,2025,2.822000e+08
872,Moonrider,Electric Tractor Tech,Bengaluru,$2.2 million (Rs 19 crore),Seed Round,"AdvantEdge Founders, Micelio Technology Fund, ...",January,2025,1.900000e+08
873,Consint.AI,Healthtech,Noida,Rs 5 crore,Funding Round,"Equanimity Ventures, Seafund",January,2025,5.000000e+07
874,NapTapGo,Affordable Hospitality,Noida,Rs 1 crore,Equity Funding,Balmer Lawrie & Co Ltd,January,2025,1.000000e+07


In [47]:

print(df_combined.columns.tolist())

['Company', 'Sector', 'Headquarters', 'Amount', 'Funding Round Type', 'Lead Investors', 'Month', 'Year', 'Amount_INR']


In [51]:


df_combined = df_combined.dropna()
df_combined.shape

(816, 9)

In [52]:
for col in df_combined.columns:
    print(f"Unique values for '{col}': {df_combined[col].nunique()}")

Unique values for 'Company': 769
Unique values for 'Sector': 638
Unique values for 'Headquarters': 77
Unique values for 'Amount': 470
Unique values for 'Funding Round Type': 132
Unique values for 'Lead Investors': 727
Unique values for 'Month': 11
Unique values for 'Year': 2
Unique values for 'Amount_INR': 323


In [54]:
# unqiue values for each couomn

for col in df_combined.columns:
    print(f"Column: '{col}'")
    print(f"  Unique Count: {df_combined[col].nunique()}")
    print(f"  Unique Values: {df_combined[col].unique()}")
    print("-" * 20)

Column: 'Company'
  Unique Count: 769
  Unique Values: ['Zaggle Prepaid Ocean Services' 'EPACK Prefab' 'Captain Fresh' 'Univest'
 'Mindgrove Technologies' 'HostBooks' 'Sparkl Edventure' 'Arata'
 'Confido Health' 'SustVest' 'Leanworx' 'Proxgy' 'Bureau' 'Bizom' 'Seekho'
 'ZingBus' 'QuiD Cash' 'Inkers Technology' 'Zunroof' 'Quanfluence' 'Mili'
 'Curie Money' 'Resilience AI' 'Fitsol' 'Fibroheal Woundcare' 'VOICE'
 'MBG Card' 'Rebel Foods' 'Mintifi' 'Zetwerk' 'CarDekho SEA' 'Haber'
 'Wooden Street' 'SolarSquare' 'K12 Techno Services' 'LambdaTest'
 'OneCard' 'Pixxel' 'Snapmint' 'RapidCanvas' 'Varthana' 'Avanti Finance'
 'Orange Health Labs' 'Stage' 'DCDC Kidney Care' 'FirstClub' 'FinX'
 'Culture Circle' 'Superfone' 'Infinity' 'Sustainiam' 'Navanc'
 'Kreedo Early Childhood Solutions' 'Power Gummies' 'SaveSage Club'
 'Nodwin Gaming' 'Atmosphere The Store Makers' 'Neuranics'
 'Annapurna Finance' 'The Money Club' 'Terractive' 'Medial'
 'Ultraviolette' 'The Indus Valley' 'Enterpret' 'ZFunds' 'Wic

In [73]:
df_combined['Lead_Investors_Clean'] = df_combined['Lead Investors'].fillna('').str.replace(r'&| and ', ',', regex=True)
df_combined['Investor_List'] = df_combined['Lead_Investors_Clean'].str.split(',').apply(lambda x: [i.strip() for i in x if i.strip()])
df_combined['num_investors'] = df_combined['Investor_List'].apply(len)




In [75]:
df_investors = df_combined.explode('Investor_List')
df_investors['Investor_List'] = df_investors['Investor_List'].str.lower().str.strip()


In [76]:
df_investors.head(2)

,Company,Sector,Headquarters,Amount,Funding Round Type,Lead Investors,Month,Year,Amount_INR,Lead_Investors_Clean,Investor_List,num_investors
0,Zaggle Prepaid Ocean Services,Enterprise Fintech,Hyderabad,Rs 595 crore,Funding Round,Qualified Institutional Placement (Bank of Ind...,December,2024,5.950000e+09,Qualified Institutional Placement (Bank of Ind...,qualified institutional placement (bank of ind...,1
1,EPACK Prefab,Pre-engineered Buildings,Greater Noida,$20 million,First Funding Round,GEF Capital Partners,December,2024,1.660000e+09,GEF Capital Partners,gef capital partners,1


In [77]:
most_active = df_investors['Investor_List'].value_counts().reset_index()
most_active.columns = ['Investor', 'Startup_Count']

In [79]:


import altair as alt
# Create a bar chart to show the 'Startup_Count' for each 'Investor'.
# Sort the bars in descending order of 'Startup_Count'.
chart = alt.Chart(most_active).mark_bar().encode(
    x=alt.X('Investor', sort='-y'), # Set 'Investor' on the x-axis and sort by 'Startup_Count'
    y='Startup_Count', # Set 'Startup_Count' on the y-axis
    tooltip=['Investor', 'Startup_Count'] # Add tooltips for interactivity
).properties(
    title='Startup Count by Investor' # Add a title to the chart
)
chart

alt.Chart(...)

In [81]:
avg_ticket = df_investors.groupby('Investor_List')['Amount_INR'].mean().reset_index()
avg_ticket.columns = ['Investor', 'Avg_Amount_INR']
avg_ticket['Avg_Amount_INR'] = avg_ticket['Avg_Amount_INR'].round(2)
avg_ticket

,Investor,Avg_Amount_INR
0,-,7.422869e+09
1,100unicorn,1.000000e+08
2,100unicorns,6.225000e+07
3,100x.vc,4.150000e+07
4,10x founders,3.700000e+08
...,...,...
1212,zeropearl vc,5.000000e+07
1213,zeropearl ventures,8.300000e+07
1214,zluri founders,1.162000e+08
1215,zoho corporation,1.300000e+09


In [85]:
sector_pref = df_investors.groupby(['Investor_List', 'Sector']).size().reset_index(name='Count')
top_sectors = (
    sector_pref.sort_values(['Investor_List', 'Count'], ascending=[True, False])
)
top_sectors

,Investor_List,Sector,Count
0,-,Agri-financing,1
1,-,Alternative Lending,1
2,-,Compostable Packaging,1
3,-,E-commerce Marketing,1
4,-,Financial Services,1
...,...,...,...
1851,zeropearl vc,Oral Care,1
1852,zeropearl ventures,Retail Luxury Goods and Jewellery,1
1853,zluri founders,SaaS AI Platform,1
1854,zoho corporation,EV Manufacturing,1
